In [1]:
import sys
sys.path.append('..')
from MPRA_exp.utils import *
from MPRA_exp.metrics import *

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.neural_network import MLPRegressor, MLPClassifier

In [2]:
def find_indice(info_df, cell_types=[], assays=[], IDs=[]):
    # 初始化索引为全部行
    indice = list(range(len(info_df)))
    
    # 如果 cell_types 不为空，则筛选 cell_type
    if cell_types:
        indice = [i for i in indice if info_df['cell_type'][i] in cell_types]
        
    # 如果 assays 不为空，则筛选 assay
    if assays:
        indice = [i for i in indice if info_df['assay'][i] in assays]
        
    # 如果 IDs 不为空，则筛选 ID
    if IDs:
        indice = [i for i in indice if info_df['ID'][i] in IDs]
    
    return indice

In [3]:
# info_df = pd.read_csv('../pretrained_models/Sei/Sei_info.csv')
# print(info_df.shape)

# pretrained_pred = np.load('../pretrained_models/Sei/Sei_Siraj_pred_3.npy')
# print(pretrained_pred.shape)

# indice = find_indice(info_df, IDs=['ENCODE', 'Roadmap'])
# info_df.iloc[indice].to_csv('../pretrained_models/Sei/Sei_info_2002.csv', index=False)
# np.save('../pretrained_models/Sei/Sei_Siraj_pred_2002.npy', pretrained_pred[:, indice])

In [4]:
info_df = pd.read_csv('../pretrained_models/Sei/Sei_info.csv')
info_df

,cell_type,assay,ID,part
0,HeLa_Epithelium_Cervix,BTAF1,ID:1,0
1,H9_Embryonic_Stem_Cell_Embryo,EOMES,ID:1006,0
2,H9_Embryonic_Stem_Cell_Embryo,EP300,ID:1007,0
3,H9_Embryonic_Stem_Cell_Embryo,FOXH1,ID:1010,0
4,H9_Embryonic_Stem_Cell_Embryo,FOXH1,ID:1011,0
...,...,...,...,...
21902,WERI-Rb-1,CTCF,ENCODE,0
21903,WERI-Rb-1,DNase,ENCODE,0
21904,WI-38,CTCF,ENCODE,0
21905,WI-38 (4OHTAM_20nM_72hr),DNase,ENCODE,0


In [5]:
# def load_npy(filename):
#     data = []
#     with open(filename, 'rb') as f:
#         while True:
#             try:
#                 d = np.load(f)
#                 data.append(d)
#             except EOFError:
#                 break
#     data = np.concatenate(data, axis=0)
#     return data

# pretrained_pred = load_npy('../pretrained_models/Sei/Sei_Siraj_pred_2.npy')
# print(pretrained_pred.shape)
# np.save('../pretrained_models/Sei/Sei_Siraj_pred_3.npy', pretrained_pred)

In [6]:
pretrained_pred = np.load('../pretrained_models/Sei/Sei_Agarwal_pred.npy')
print(pretrained_pred.shape)

main_df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/AgarwalMPRA/Agarwal.csv')
print(main_df.shape)


(56975, 21907)
(56975, 5)


In [7]:
# pretrained_pred = np.load('../pretrained_models/Sei/Sei_Siraj_pred.npy')
# print(pretrained_pred.shape)

# pred = np.load('../pretrained_models/Sei/Sei_Gosai_designed_pred.npy')
# pretrained_pred = np.concatenate([pretrained_pred, pred])
# print(pretrained_pred.shape)


# main_df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/SirajMPRA_total.csv')
# print(main_df.shape)

# df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/GosaiMPRA/GosaiMPRA_designed.csv')
# df['chr'] = 'chr0'
# main_df = pd.concat([main_df, df]).reset_index(drop=True)
# print(main_df.shape)

In [8]:
# # 定义细胞所有可能的名字
aliases_dict = {
    'HepG2': [
        r"hepg2",
        r"hepatocellular[_\s-]*carcinoma",
        r"hep[_\s-]*g[_\s-]*2",
        ],
    'K562': [
        r"K562",
    ],
    'SKNSH': [
        r"neuroblastoma",
        r"SKNSH",
        r"SK-N-SH",
    ],
    'A549': [
        r"A549",
    ],
    'HCT116': [
        r"HCT116",
        r"colon",
    ],
    'WTC11': [
        r"WTC11",
        r"iPS",
        r"GM23338",
    ],
}


assays = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27me3', 'H3K27ac', 'H3K36me3', 'CTCF']
# cell_types = ['HepG2', 'K562', 'SKNSH', 'A549', 'HCT116']
cell_types = ['HepG2', 'K562', 'WTC11']
info_df['pearson_r'] = np.nan

for cell_type in cell_types:
    aliases = aliases_dict[cell_type]
    pattern = re.compile("|".join(aliases), re.IGNORECASE) # 不区分大小写
    matched_indice = [i for i in range(len(info_df))
                      if (pattern.search(info_df['cell_type'][i])) and (info_df['assay'][i] in assays)]
    
    for i in matched_indice:
        x = logit(pretrained_pred[:, i])
        y = main_df[cell_type]
        r = pearson(x, y)
        info_df.loc[i, 'pearson_r'] = r

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 10000)
    print(info_df.iloc[matched_indice].sort_values(by='cell_type'))
    pd.set_option('display.max_rows', 10)
    cell_type_names = info_df.iloc[matched_indice]['cell_type']
    counter = Counter(cell_type_names)
    for name in counter:
        print(f'{name}: {counter[name]}')

                            cell_type     assay        ID  part  pearson_r
20977                           HepG2     DNase    ENCODE     0   0.290027
20976                           HepG2      CTCF    ENCODE     3   0.031598
20975                           HepG2      CTCF    ENCODE     2  -0.011794
20974                           HepG2      CTCF    ENCODE     1  -0.000678
20973                           HepG2      CTCF    ENCODE     0   0.055424
144            HepG2_Epithelium_Liver      CTCF   ID:1380     0  -0.038005
4850           HepG2_Epithelium_Liver      CTCF  ID:45303     0   0.015019
4852           HepG2_Epithelium_Liver   H3K4me3  ID:45306     0   0.333355
5311           HepG2_Epithelium_Liver      CTCF  ID:45794     0  -0.009564
5312           HepG2_Epithelium_Liver      CTCF  ID:45795     0  -0.003827
5325           HepG2_Epithelium_Liver      CTCF  ID:45808     0  -0.059209
5765           HepG2_Epithelium_Liver      CTCF  ID:46250     0  -0.025895
7480           HepG2_Epit

In [13]:
# # 不一定全
# cell_types_names_dict = {
#     'HepG2': ['HepG2', 'HepG2_Epithelium_Liver', 'HepG2_Hepatocellular_Carcinoma'],
#     'K562' : ['K562', 'K562_Erythroblast_Bone_Marrow'],
#     'SKNSH': ['SK-N-SH_RA_Neuron_Brain', 'Neuroblastoma_cell', 'SK-N-SH_Neuroblastoma_cell_Brain'],
#     'A549' : ['A549_Epithelium_Lung'],
#     'HCT116':['HCT-116_HCT116', 'HCT-116_Colon'],
#     'WTC11': ['iPSC'],
# }


cell_types_names_dict = {
    'HepG2': ['HepG2_Epithelium_Liver'],
    'K562' : ['K562_Erythroblast_Bone_Marrow'],
    'SKNSH': ['SK-N-SH_Neuroblastoma_cell_Brain'],
    'A549' : ['A549_Epithelium_Lung'],
    'HCT116':['HCT-116_Colon'],
    'WTC11': ['iPSC'],
}


warnings.filterwarnings("ignore", category=RuntimeWarning)

for cell_type in cell_types:
    for assay in assays:
        cell_names = cell_types_names_dict[cell_type]
        matched_indice = find_indice(info_df, cell_names, [assay])
        value = logit(pretrained_pred[:, matched_indice]).mean(axis=1) #先logit再mean
        main_df[f'{cell_type}_{assay}_pred_logit'] = value

        y = main_df[cell_type]
        r = pearson(value, y)
        print(f'{cell_type} {assay}, found {len(matched_indice)} records, pearson r = {r:.3f}')

main_df = main_df.copy()

HepG2 DNase, found 7 records, pearson r = 0.368
HepG2 H3K4me1, found 3 records, pearson r = 0.386
HepG2 H3K4me3, found 7 records, pearson r = 0.387
HepG2 H3K9me3, found 1 records, pearson r = -0.050
HepG2 H3K27me3, found 1 records, pearson r = 0.113
HepG2 H3K27ac, found 8 records, pearson r = 0.404
HepG2 H3K36me3, found 2 records, pearson r = 0.262
HepG2 CTCF, found 13 records, pearson r = -0.014
K562 DNase, found 56 records, pearson r = 0.483
K562 H3K4me1, found 9 records, pearson r = 0.404
K562 H3K4me3, found 90 records, pearson r = 0.496
K562 H3K9me3, found 20 records, pearson r = 0.268
K562 H3K27me3, found 33 records, pearson r = 0.101
K562 H3K27ac, found 50 records, pearson r = 0.472
K562 H3K36me3, found 4 records, pearson r = 0.352
K562 CTCF, found 22 records, pearson r = 0.098
WTC11 DNase, found 5 records, pearson r = 0.477
WTC11 H3K4me1, found 8 records, pearson r = 0.276
WTC11 H3K4me3, found 13 records, pearson r = 0.431
WTC11 H3K9me3, found 8 records, pearson r = 0.223
WTC11 

In [14]:
main_df.to_csv('data/Sei_Agarwal_features.csv', index=False)